<a href="https://colab.research.google.com/github/AmilcarArmmand/CSC-481/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

import requests
import io

base_URL = 'https://raw.githubusercontent.com/AmilcarArmmand/CSC-481/refs/heads/main/Face_Database/'
URL = 'https://raw.githubusercontent.com/AmilcarArmmand/CSC-481/refs/heads/main/Face_Database/m-001/m-001-01.pts'


def compute_distance(df, point1, point2):
    """Calculate Euclidean distance between two points."""
    x1, x2 = df.loc[point1, 'X'], df.loc[point2, 'X']
    y1, y2 = df.loc[point1, 'Y'], df.loc[point2, 'Y']
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

'''
# Read the file skipping
df = pd.read_table(URL, sep=' ', header=None, skiprows=3)
# Drop the last line
df = df.drop(df.index[-1:])
df = df.rename(columns={0: 'X', 1: 'Y'})

df['X'] = pd.to_numeric(df['X'])
df['Y'] = pd.to_numeric(df['Y'])
print(df)
df.dtypes
'''

"\n# Read the file skipping\ndf = pd.read_table(URL, sep=' ', header=None, skiprows=3)\n# Drop the last line\ndf = df.drop(df.index[-1:])\ndf = df.rename(columns={0: 'X', 1: 'Y'})\n\ndf['X'] = pd.to_numeric(df['X'])\ndf['Y'] = pd.to_numeric(df['Y'])\nprint(df)\ndf.dtypes\n"

Feature definitions
1.	Eye length ratio: length of eye (maximum of two) over distance between points 8 and 13.
2.	Eye distance ratio: distance between center of two eyes over distance between points 8 and 13.
3.	Nose ratio: Distance between points 15 and 16 over distance between 20 and 21.
4.	Lip size ratio: Distance between points 2 and 3 over distance between 17 and 18.
5.	Lip length ratio: Distance between points 2 and 3 over distance between 20 and 21.
6.	Eye-brow length ratio: Distance between points 4 and 5 (or distance between points 6 and 7 whichever is larger) over distance between 8 and 13.
7.	Aggressive ratio: Distance between points 10 and 19 over distance between 20 and 21.


In [2]:
'''
features = {}

# Feature 1: Eye length ratio, length of eye (maximum of two) over distance between points 8 and 13.
left_eye = compute_distance(df, 9, 10)
right_eye = compute_distance(df, 11, 12)
denom1 = compute_distance(df, 8, 13)
features['Eye length ratio'] = max(left_eye, right_eye) / denom1 if denom1 else 0

# Feature 2: Eye distance ratio - distance between center of two eyes over distance between points 8 and 13
eye_dist = compute_distance(df, 0, 1)
head_dist = compute_distance(df, 8, 13)
features['Eye distance ratio'] = eye_dist / head_dist if head_dist else 0

# Feature 3: Nose ratio - Distance between points 15 and 16 over distance between 20 and 21.
nose_num = compute_distance(df, 15, 16)
nose_den = compute_distance(df, 20, 21)
features['Nose ratio'] = nose_num / nose_den if nose_den else 0

# Feature 4: Lip size ratio - Distance between points 2 and 3 over distance between 17 and 18.
lip_num = compute_distance(df, 2, 3)
lip_den = compute_distance(df, 17, 18)
features['Lip size ratio'] = lip_num / lip_den if lip_den else 0

# Features 5: Lip length ratio - Distance between points 2 and 3 over distance between 20 and 21.
lip_num = compute_distance(df, 2, 3)
lip_den = compute_distance(df, 20, 21)
features['Lip length ratio'] = lip_num / lip_den if lip_den else 0

# Feature 6: Eye-brow length ratio - Distance between points 4 and 5 (or distance between points 6 and 7 whichever is larger) over distance between 8 and 13.
brow_num = max(compute_distance(df, 4, 5), compute_distance(df, 6, 7))
features['Eye-brow length ratio'] = brow_num / denom1 if denom1 else 0

# Feature 7: Aggressive ratio - Distance between points 10 and 19 over distance between 20 and 21
aggro_num = compute_distance(df, 10, 19)
features['Aggressive Ratio'] = aggro_num / nose_den if nose_den else 0


print(features)
'''

{'Eye length ratio': 0.19330713710291383, 'Eye distance ratio': 0.4237684655772815, 'Nose ratio': 0.1660082668499721, 'Lip size ratio': 4.474275910905935, 'Lip length ratio': 0.4361737683401592, 'Eye-brow length ratio': 0.4267253292112548, 'Aggressive Ratio': 0.9445179202545555}


In [7]:
def extract_features(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_table(io.StringIO(response.text), sep=' ', header=None, skiprows=3)
        df = df.drop(df.index[-1:])
        df = df.rename(columns={0: 'X', 1: 'Y'})
        df['X'] = pd.to_numeric(df['X'])
        df['Y'] = pd.to_numeric(df['Y'])
    except Exception as e:
        print(f"Error: {e}")
        return None

    if len(df) != 22:
        print("Invalid number of points. Expected 22, got {:} for {}".format(len(df)), url)
        return None

    features = {}

    # Feature 1: Eye length ratio, length of eye (maximum of two) over distance between points 8 and 13.
    left_eye = compute_distance(df, 9, 10)
    right_eye = compute_distance(df, 11, 12)
    denom1 = compute_distance(df, 8, 13)
    features['Eye Length Ratio'] = max(left_eye, right_eye) / denom1 if denom1 else 0

    # Feature 2: Eye distance ratio - distance between center of two eyes over distance between points 8 and 13
    eye_dist = compute_distance(df, 0, 1)
    head_dist = compute_distance(df, 8, 13)
    features['Eye Distance Ratio'] = eye_dist / head_dist if head_dist else 0

    # Feature 3: Nose ratio - Distance between points 15 and 16 over distance between 20 and 21.
    nose_num = compute_distance(df, 15, 16)
    nose_den = compute_distance(df, 20, 21)
    features['Nose Ratio'] = nose_num / nose_den if nose_den else 0

    # Feature 4: Lip size ratio - Distance between points 2 and 3 over distance between 17 and 18.
    lip_num = compute_distance(df, 2, 3)
    lip_den = compute_distance(df, 17, 18)
    features['Lip Size Ratio'] = lip_num / lip_den if lip_den else 0

    # Features 5: Lip length ratio - Distance between points 2 and 3 over distance between 20 and 21.
    lip_num = compute_distance(df, 2, 3)
    lip_den = compute_distance(df, 20, 21)
    features['Lip Length Ratio'] = lip_num / lip_den if lip_den else 0

    # Feature 6: Eye-brow length ratio - Distance between points 4 and 5 (or distance between points 6 and 7 whichever is larger) over distance between 8 and 13.
    brow_num = max(compute_distance(df, 4, 5), compute_distance(df, 6, 7))
    features['Eye-brow Length Ratio'] = brow_num / denom1 if denom1 else 0

    # Feature 7: Aggressive ratio - Distance between points 10 and 19 over distance between 20 and 21
    aggro_num = compute_distance(df, 10, 19)
    features['Aggressive Ratio'] = aggro_num / nose_den if nose_den else 0

    return(features)


In [8]:
def main():
    base_url = 'https://raw.githubusercontent.com/AmilcarArmmand/CSC-481/main/Face_Database/'
    id_list = [1, 2, 3, 4, 5]
    img_list = [1, 2, 3, 5]
    persons = [f'm-{i:03d}' for i in id_list] + [f'w-{i:03d}' for i in id_list]
    results = []

    for person in persons:
        for img_num in img_list:
            url = f"{base_url}{person}/{person}-{img_num:02d}.pts"
            features = extract_features(url)
            if features:
                results.append({
                    'Person': person,
                    'Image': img_num,
                    **features
                })

    df = pd.DataFrame(results)
    print("Feature extraction complete. Sample output:")
    print(df)

if __name__ == "__main__":
    main()


Feature extraction complete. Sample output:
   Person  Image  Eye Length Ratio  Eye Distance Ratio  Nose Ratio  \
0   m-001      1          0.193307            0.423768    0.166008   
1   m-001      2          0.211407            0.428316    0.147227   
2   m-001      3          0.224320            0.434374    0.155068   
3   m-001      5          0.183316            0.445299    0.162857   
4   m-002      1          0.208663            0.480300    0.135395   
5   m-002      2          0.233480            0.487658    0.135987   
6   m-002      3          0.234001            0.482573    0.134480   
7   m-002      5          0.182113            0.490274    0.138033   
8   m-003      1          0.239009            0.468884    0.147836   
9   m-003      2          0.241601            0.472005    0.163104   
10  m-003      3          0.241439            0.475170    0.160229   
11  m-003      5          0.176839            0.484628    0.172419   
12  m-004      1          0.236852            